In [7]:
# 确定当前路径, 方便指定词典路径
import os
!cd

N:\00-work\ws_ne_seg\negseg-tutorial\ipynb


In [8]:
import pandas as pd

DICT_DIR='..\workspace\output-dict'
DICT_SRC_DIR='..\workspace\input-dict'

# 自定义: 国名前缀,企业名称后缀,部分地名自定义
excel_dict = '%s\设备供应商-词典.xlsx' % DICT_SRC_DIR

# 行政区划标准库
excel_name_dmk = u'%s\\2017年最新行政区划数据库.xls' % DICT_SRC_DIR

## 国名前缀

In [9]:
# 国名前缀
csv_name_country = '%s\dict-country.csv' % DICT_DIR

df = pd.read_excel(excel_dict,sheet_name='国名前缀',skiprows=0,index=False,encoding='utf8')
df.drop_duplicates(inplace=True)
df.dropna(axis=0,inplace=True)
df.eval('count = Name.str.len()',inplace=True)
df.sort_values(by=['count'], inplace = True, ascending=False)
df.to_csv(csv_name_country,index=False,header=False,sep=',',encoding='utf-8')

## 品类行业+名称后缀

In [10]:
csv_name_suffix = '%s\dict-suffix.csv' % DICT_DIR

df1 = pd.read_excel(excel_dict,sheet_name='品类行业',skiprows=0,usecols=[0],index=False,encoding='utf8')
df2 = pd.read_excel(excel_dict,sheet_name='名称后缀',skiprows=0,usecols=[0],index=False,encoding='utf8')
df = pd.concat([df1,df2],axis=0)

df.drop_duplicates(inplace=True)
df.dropna(axis=0,inplace=True)
df.eval('count = Name.str.len()',inplace=True)
df.sort_values(by=['count'], inplace = True, ascending=False)
df.to_csv(csv_name_suffix,index=False,encoding='utf-8')

## 全部行政机构

In [11]:


df = pd.read_excel(excel_name_dmk,sheet_name='旗舰版',skiprows=0,usecols=[3,4,5,7],encoding='utf8')

def getDict(dfDMK, levelType, excelDict, sheetName):
    df_xz = dfDMK[dfDMK['LevelType'] == levelType]
    df_xz_ts = pd.read_excel(excelDict,sheet_name=sheetName,skiprows=0,usecols=[0,1],encoding='utf8')

    df_xz = pd.concat([df_xz,df_xz_ts],axis=0,sort=False)

    namelist = df_xz['ShortName'].tolist()+df_xz['Name'].tolist()+df_xz['NameAlias'].tolist()
    df_xz = pd.DataFrame({'Name':namelist})
    df_xz.drop_duplicates(inplace=True)
    df_xz.dropna(axis=0,inplace=True)
    df_xz.eval('Len = Name.str.len()',inplace=True)
    df_xz.sort_values(by=['Len'],ascending=False,inplace=True)

    return df_xz

## 行政一级+二级+三级生成词典

In [12]:
csv_name_xz1 = '%s\dict-province.csv' % DICT_DIR
csv_name_xz2 = '%s\dict-city.csv' % DICT_DIR
csv_name_xz3 = '%s\dict-county.csv' % DICT_DIR

df_xz1 = getDict(df, 1, excel_dict, sheetName='行政一级特殊')
df_xz1.to_csv(csv_name_xz1,index=False,header=False,sep=',',encoding='utf-8')

df_xz2 = getDict(df, 2, excel_dict, sheetName='行政二级特殊')
df_xz2.to_csv(csv_name_xz2,index=False,header=False,sep=',',encoding='utf-8')

df_xz3 = getDict(df, 3, excel_dict, sheetName='行政三级特殊')
df_xz3.to_csv(csv_name_xz3,index=False,header=False,sep=',',encoding='utf-8')